In [1]:
import numpy as np
import pandas as pd
import holidays
from datetime import timedelta, date

In [2]:
data = pd.read_pickle("preparedData.pkl")
data

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,combinedSpaceStationID,timeConnected,idleTime,chargingTime,weekday,month,time,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,194.0,AG-3F30/1-1-179-810,21741.0,5980.0,15761.0,3,1,13:08:54,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,4275.0,AG-1F01/1-1-193-825,32491.0,8416.0,24075.0,3,1,13:36:50,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,344.0,AG-1F03/1-1-193-829,38567.0,29056.0,9511.0,3,1,13:56:35,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,1117.0,AG-1F04/1-1-193-820,9521.0,4794.0,4727.0,3,1,13:59:58,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,334.0,AG-1F06/1-1-193-819,29319.0,13870.0,15449.0,3,1,14:00:01,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56531,2019-07-01 19:13:11+00:00,2019-07-02 00:44:40+00:00,2019-07-01 21:13:56+00:00,6.048,1,2643.0,AG-3F23/1-1-179-797,19889.0,12644.0,7245.0,0,7,19:13:11,275.0,8.25,30.0,374.0,"Mon, 01 Jul 2019 19:13:10 GMT",True,"Tue, 02 Jul 2019 01:27:11 GMT"
56532,2019-07-01 19:43:46+00:00,2019-07-02 02:58:53+00:00,2019-07-02 02:43:53+00:00,41.047,1,394.0,AG-4F38/1-1-191-793,26107.0,900.0,25207.0,0,7,19:43:46,231.0,36.96,160.0,310.0,"Mon, 01 Jul 2019 19:43:58 GMT",True,"Tue, 02 Jul 2019 00:53:46 GMT"
56533,2019-07-01 20:34:30+00:00,2019-07-02 00:30:52+00:00,2019-07-01 21:27:44+00:00,5.390,1,2276.0,AG-3F15/1-1-179-813,14182.0,10988.0,3194.0,0,7,20:34:30,300.0,12.00,40.0,381.0,"Mon, 01 Jul 2019 20:34:29 GMT",True,"Tue, 02 Jul 2019 02:55:30 GMT"
56534,2019-07-01 21:49:12+00:00,2019-07-02 01:37:18+00:00,2019-07-01 23:14:33+00:00,8.399,1,1346.0,AG-3F18/1-1-179-798,13686.0,8565.0,5121.0,0,7,21:49:12,400.0,32.00,80.0,36.0,"Mon, 01 Jul 2019 21:49:37 GMT",True,"Mon, 01 Jul 2019 22:25:12 GMT"


For utilization prediction we will exclude any data after March 2020 due to the Corona pandemic. We argue, that due to lockdowns and a shift to homeoffice whereever possible, the rest of 2020 is not relevant to the situation we have today. While homeoffice is generally more accepted nowadays, we are beginning to see a lot of companies enforcing a return to the office.
While the data is beginning to normalize at the beginning of 2021, we do not have any weather data after this point. Attempts to find additional data for the burbank airport where unsuccessful. Due to both of these points, there will be a hard cutoff after March 2020.

In [3]:
data_nc = data[data['disconnectTime']<"2020-4-1"].copy()

In [4]:
#Split data into sites 1 and 2
site_1_data = data_nc[data_nc['siteID']==1].copy()
site_2_data = data_nc[data_nc['siteID']==2].copy()

In [5]:
site_1_data.info()
site_2_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21233 entries, 0 to 56535
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   connectionTime          21233 non-null  datetime64[ns, UTC]
 1   disconnectTime          21233 non-null  datetime64[ns, UTC]
 2   doneChargingTime        21233 non-null  datetime64[ns, UTC]
 3   kWhDelivered            21233 non-null  float64            
 4   siteID                  21233 non-null  int64              
 5   userID                  20119 non-null  float64            
 6   combinedSpaceStationID  21233 non-null  object             
 7   timeConnected           21233 non-null  float64            
 8   idleTime                21233 non-null  float64            
 9   chargingTime            21233 non-null  float64            
 10  weekday                 21233 non-null  int32              
 11  month                   21233 non-null  int32 

In [6]:
site_1_data.reset_index(inplace=True)
site_2_data.reset_index(inplace=True)

In [7]:
# Drop the siteID, it's no longer needed
site_1_data.drop(columns=['siteID'], inplace=True)
site_2_data.drop(columns=['siteID'], inplace=True)

In [8]:
# Check for unique space/station id's so we know the maximum number of parking spaces available
site_1_data.nunique()

index                     21233
connectionTime            21135
disconnectTime            21207
doneChargingTime          21105
kWhDelivered              14133
userID                      406
combinedSpaceStationID       52
timeConnected             12429
idleTime                  13909
chargingTime              14438
weekday                       7
month                        12
time                      13982
WhPerMile                   106
kWhRequested                691
milesRequested               54
minutesAvailable            773
modifiedAt                20033
paymentRequired               1
requestedDeparture        20100
dtype: int64

In [9]:
site_2_data.nunique()

index                     26046
connectionTime            26008
disconnectTime            26011
doneChargingTime          25930
kWhDelivered              13877
userID                      395
combinedSpaceStationID       54
timeConnected             17042
idleTime                  12635
chargingTime              13262
weekday                       7
month                        12
time                      19630
WhPerMile                    78
kWhRequested                494
milesRequested               48
minutesAvailable            665
modifiedAt                12335
paymentRequired               1
requestedDeparture        12339
dtype: int64

52 spaces for site 1, 54 for site 2.

In [10]:
site_1_data["connectionTime"].describe()

count                                  21233
mean     2019-06-18 14:54:33.518720768+00:00
min                2018-10-08 13:01:34+00:00
25%                2019-02-12 23:52:16+00:00
50%                2019-06-05 02:48:15+00:00
75%                2019-10-22 13:48:21+00:00
max                2020-03-31 13:54:38+00:00
Name: connectionTime, dtype: object

In [11]:
site_1_data["disconnectTime"].describe()

count                                  21233
mean     2019-06-18 22:30:34.762021376+00:00
min                2018-10-09 00:05:33+00:00
25%                2019-02-13 02:02:37+00:00
50%                2019-06-05 09:54:38+00:00
75%                2019-10-22 23:03:46+00:00
max                2020-03-31 23:45:16+00:00
Name: disconnectTime, dtype: object

In [12]:
site_2_data["connectionTime"].describe()

count                                  26046
mean     2019-01-18 15:54:24.067572992+00:00
min                2018-04-25 13:45:10+00:00
25%      2018-08-13 05:22:48.750000128+00:00
50%                2018-11-14 19:57:07+00:00
75%                2019-06-12 16:06:28+00:00
max                2020-03-29 00:40:26+00:00
Name: connectionTime, dtype: object

In [13]:
site_2_data["disconnectTime"].describe()

count                                  26046
mean     2019-01-18 21:54:36.511095552+00:00
min                2018-04-25 18:15:46+00:00
25%      2018-08-13 13:51:00.249999872+00:00
50%                2018-11-15 02:56:54+00:00
75%      2019-06-12 23:25:12.249999872+00:00
max                2020-03-29 14:43:45+00:00
Name: disconnectTime, dtype: object

In [14]:
# Create a dataframe with dates from the first to the last day of the charging sessions available. Because these differ for both sites, we need to do this for each respectively. Add a timedelta of 12 hours to site 2 disconnect, otherwise it would cut off the last day due to the hour of the timestamp
date_frame_site_1 = pd.DataFrame(pd.date_range(start=site_1_data["connectionTime"].min(), end = site_1_data["disconnectTime"].max()))
date_frame_site_2 = pd.DataFrame(pd.date_range(start=site_2_data["connectionTime"].min(), end = (site_2_data["disconnectTime"].max() + timedelta(hours = 12))))

In [15]:
date_frame_site_1

,0
0,2018-10-08 13:01:34+00:00
1,2018-10-09 13:01:34+00:00
2,2018-10-10 13:01:34+00:00
3,2018-10-11 13:01:34+00:00
4,2018-10-12 13:01:34+00:00
...,...
536,2020-03-27 13:01:34+00:00
537,2020-03-28 13:01:34+00:00
538,2020-03-29 13:01:34+00:00
539,2020-03-30 13:01:34+00:00


In [16]:
data = {'date': [], 'hour': []}
for row in date_frame_site_1.itertuples():
    for i in range(24):
        data["date"].append(row[1])
        data["hour"].append(i)
frame1 = pd.DataFrame(data)
frame1["weekday"] = frame1["date"].dt.day_of_week
frame1["month"] = frame1["date"].dt.month

data = {'date' : [], 'hour': []}
for row in date_frame_site_2.itertuples():
    for i in range(24):
        data["date"].append(row[1])
        data["hour"].append(i)
frame2 = pd.DataFrame(data)
frame2["weekday"] = frame2["date"].dt.day_of_week
frame2["month"] = frame2["date"].dt.month

In [17]:
frame2

,date,hour,weekday,month
0,2018-04-25 13:45:10+00:00,0,2,4
1,2018-04-25 13:45:10+00:00,1,2,4
2,2018-04-25 13:45:10+00:00,2,2,4
3,2018-04-25 13:45:10+00:00,3,2,4
4,2018-04-25 13:45:10+00:00,4,2,4
...,...,...,...,...
16915,2020-03-29 13:45:10+00:00,19,6,3
16916,2020-03-29 13:45:10+00:00,20,6,3
16917,2020-03-29 13:45:10+00:00,21,6,3
16918,2020-03-29 13:45:10+00:00,22,6,3


In [18]:
us_ca_holidays = holidays.country_holidays(country='US', subdiv='CA')

frame1["holiday"] = frame1["date"].apply(lambda x: x in us_ca_holidays)
frame2["holiday"] = frame2["date"].apply(lambda x: x in us_ca_holidays)

Since the data doesn't start at the beginning of the day and doesn't end at the end, we'll manually delete a few rows to fit it properly. 
Both sites start in hour 13. Site 1 ends in hour 14 so we will have to drop the last 9 rows.
 Site 2 ends in hour 2, so we will drop the last 21.

In [19]:
frame1.drop(frame1.index[:13], inplace=True)
frame2.drop(frame2.index[:13], inplace=True)
frame2.drop(frame2.index[-9:], inplace=True)

In [20]:
weather = pd.read_csv("weather_burbank_airport.csv")
weather

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...
29239,Burbank,2021-01-01 03:53:00,13.0,33.0,Fair,986.81,0.0,0.0,13.0
29240,Burbank,2021-01-01 04:53:00,12.0,33.0,Fair,986.81,11.0,0.0,12.0
29241,Burbank,2021-01-01 05:53:00,12.0,33.0,Fair,987.47,9.0,0.0,12.0
29242,Burbank,2021-01-01 06:53:00,11.0,33.0,Fair,987.14,13.0,0.0,11.0


In [21]:
# We will only use temperature and precipitation from the weather dataset, so we'll drop everything else
weather.drop(columns=["city", 'cloud_cover', 'cloud_cover_description', 'pressure', 'windspeed', 'felt_temperature'], axis = 1, inplace=True)

In [22]:
weather

,timestamp,temperature,precipitation
0,2018-01-01 08:53:00,9.0,0.0
1,2018-01-01 09:53:00,9.0,0.0
2,2018-01-01 10:53:00,9.0,0.0
3,2018-01-01 11:53:00,9.0,0.0
4,2018-01-01 12:53:00,8.0,0.0
...,...,...,...
29239,2021-01-01 03:53:00,13.0,0.0
29240,2021-01-01 04:53:00,12.0,0.0
29241,2021-01-01 05:53:00,12.0,0.0
29242,2021-01-01 06:53:00,11.0,0.0


In [23]:
weather['timestamp'] = pd.to_datetime(weather['timestamp'])

In [24]:
weather['timestamp'].dt.date

0        2018-01-01
1        2018-01-01
2        2018-01-01
3        2018-01-01
4        2018-01-01
            ...    
29239    2021-01-01
29240    2021-01-01
29241    2021-01-01
29242    2021-01-01
29243    2021-01-01
Name: timestamp, Length: 29244, dtype: object

In [25]:
# Add a new column dateHour, which represents the date with the current hour, we will merge into the site data on this later on. Drop the timestamp because we don't have any more use for it
weather["dateHour"] = weather["timestamp"].apply(lambda x: (str(x.year) + "-"
                                                            + str(x.month) +"-"
                                                            + str(x.day) +"+"
                                                            + str(x.hour)))
weather.drop(columns="timestamp")

,temperature,precipitation,dateHour
0,9.0,0.0,2018-1-1+8
1,9.0,0.0,2018-1-1+9
2,9.0,0.0,2018-1-1+10
3,9.0,0.0,2018-1-1+11
4,8.0,0.0,2018-1-1+12
...,...,...,...
29239,13.0,0.0,2021-1-1+3
29240,12.0,0.0,2021-1-1+4
29241,12.0,0.0,2021-1-1+5
29242,11.0,0.0,2021-1-1+6


In [26]:
# Add the dateHour column to both dataframes
frame1["dateHour"] = frame1["date"].apply(lambda x: (str(x.year) + "-"
                                                            + str(x.month) +"-"
                                                            + str(x.day) +"+"
                                                            ))
frame1["dateHour"] = frame1.apply(lambda x: '%s%s' % (x["dateHour"], x["hour"]), axis =1)

frame2["dateHour"] = frame2["date"].apply(lambda x: (str(x.year) + "-"
                                                            + str(x.month) +"-"
                                                            + str(x.day) +"+"
                                                            ))
frame2["dateHour"] = frame2.apply(lambda x: '%s%s' % (x["dateHour"], x["hour"]), axis =1)

In [27]:
# Function to return all hours between a start and end date in dateHour notation
def getHoursBetween(start, end):
    start = start.floor("h")
    end = end.floor("h")
    days= []
    for i in range(int((end-start).seconds/3600)+1):
            day = start+timedelta(hours=i)
            dateHour = str(day.year) + "-" + str(day.month) + "-" + str(day.day) + "+" + str(day.hour)
            days.append(dateHour)
    return days

In [28]:
# Iterate over all site 1 data, for each charging event, increment utilization by one for each hour. Keep track of the charging stations in stations, each station can have a max utilization of 1, meaning there is no difference if the station was occupied for 1 or 60 hour, or if one charging event ended and another started in the same hour
frame1["utilization"] = 0
frame1["stations"] = ""

for row in site_1_data.itertuples():
    connectionTime = row[2]
    disconnectTime = row[3]
    station = row[7]
    hours = getHoursBetween(connectionTime, disconnectTime)
    for hour in hours:
        hourRowIndex = frame1[frame1["dateHour"] == hour].index.tolist()[0]
        if not (str(frame1.at[hourRowIndex, "stations"]).__contains__(station)):
            frame1.at[hourRowIndex, "utilization"] += 1
            frame1.at[hourRowIndex, "stations"] += station +"_"
frame1['utilization'] /= 52
frame1.drop(columns="stations", inplace=True)

In [29]:
# Iterate over all site 2 data, for each charging event, increment utilization by one for each hour. Keep track of the charging stations in stations, each station can have a max utilization of 1, meaning there is no difference if the station was occupied for 1 or 60 hour, or if one charging event ended and another started in the same hour
frame2["utilization"] = 0
frame2["stations"] = ""

for row in site_2_data.itertuples():
    connectionTime = row[2]
    disconnectTime = row[3]
    station = row[7]
    hours = getHoursBetween(connectionTime, disconnectTime)
    for hour in hours:
        hourRowIndex = frame2[frame2["dateHour"] == hour].index.tolist()[0]
        if not (str(frame2.at[hourRowIndex, "stations"]).__contains__(station)):
            frame2.at[hourRowIndex, "utilization"] += 1
            frame2.at[hourRowIndex, "stations"] += station +"_"
frame2['utilization'] /= 54
frame2.drop(columns="stations", inplace = True)

In [30]:
# drop duplicates in weather to ensure correct merge of both dataframes
weather.drop_duplicates(subset='dateHour', inplace=True)

In [31]:
# Merge weather data with the charging data
frame1 = frame1.merge(weather, left_on="dateHour", right_on="dateHour", how="left")
frame2 = frame2.merge(weather, left_on="dateHour", right_on="dateHour", how="left")

In [35]:
frame1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12971 entries, 0 to 12970
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   date           12971 non-null  datetime64[ns, UTC]
 1   hour           12971 non-null  int64              
 2   weekday        12971 non-null  int32              
 3   month          12971 non-null  int32              
 4   holiday        12971 non-null  bool               
 5   dateHour       12971 non-null  object             
 6   utilization    12971 non-null  float64            
 7   timestamp      12947 non-null  datetime64[ns]     
 8   temperature    12938 non-null  float64            
 9   precipitation  12947 non-null  float64            
dtypes: bool(1), datetime64[ns, UTC](1), datetime64[ns](1), float64(3), int32(2), int64(1), object(1)
memory usage: 823.5+ KB


In [36]:
frame2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16898 entries, 0 to 16897
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   date           16898 non-null  datetime64[ns, UTC]
 1   hour           16898 non-null  int64              
 2   weekday        16898 non-null  int32              
 3   month          16898 non-null  int32              
 4   holiday        16898 non-null  bool               
 5   dateHour       16898 non-null  object             
 6   utilization    16898 non-null  float64            
 7   timestamp      16858 non-null  datetime64[ns]     
 8   temperature    16848 non-null  float64            
 9   precipitation  16858 non-null  float64            
dtypes: bool(1), datetime64[ns, UTC](1), datetime64[ns](1), float64(3), int32(2), int64(1), object(1)
memory usage: 1.0+ MB


In [34]:
frame1

,date,hour,weekday,month,holiday,dateHour,utilization,timestamp,temperature,precipitation
0,2018-10-08 13:01:34+00:00,13,0,10,True,2018-10-8+13,0.076923,2018-10-08 13:53:00,13.0,0.0
1,2018-10-08 13:01:34+00:00,14,0,10,True,2018-10-8+14,0.365385,2018-10-08 14:20:00,14.0,0.0
2,2018-10-08 13:01:34+00:00,15,0,10,True,2018-10-8+15,0.365385,2018-10-08 15:53:00,17.0,0.0
3,2018-10-08 13:01:34+00:00,16,0,10,True,2018-10-8+16,0.365385,2018-10-08 16:53:00,18.0,0.0
4,2018-10-08 13:01:34+00:00,17,0,10,True,2018-10-8+17,0.365385,2018-10-08 17:53:00,20.0,0.0
...,...,...,...,...,...,...,...,...,...,...
12966,2020-03-31 13:01:34+00:00,19,1,3,True,2020-3-31+19,0.057692,2020-03-31 19:53:00,23.0,0.0
12967,2020-03-31 13:01:34+00:00,20,1,3,True,2020-3-31+20,0.057692,2020-03-31 20:53:00,24.0,0.0
12968,2020-03-31 13:01:34+00:00,21,1,3,True,2020-3-31+21,0.057692,2020-03-31 21:53:00,25.0,0.0
12969,2020-03-31 13:01:34+00:00,22,1,3,True,2020-3-31+22,0.057692,2020-03-31 22:53:00,24.0,0.0


There are a few datapoints where we dont have the weather data available. Because we are working with weather data, we don't expect things to change abruptly. Temperature will likely drop from 14->12->10 instead of 14->10->10. We will use pandas interpolate to model this change for missing values.

In [37]:
frame1.interpolate(inplace=True)
frame2.interpolate(inplace=True)

C:\Users\Ann\AppData\Local\Temp\ipykernel_22696\2997595545.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  frame1.interpolate(inplace=True)
C:\Users\Ann\AppData\Local\Temp\ipykernel_22696\2997595545.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  frame2.interpolate(inplace=True)


In [40]:
frame1.drop(columns=['date','dateHour','timestamp'], inplace=True)
frame2.drop(columns=['date','dateHour','timestamp'], inplace=True)

In [46]:
frame1.to_pickle('./utilization_data_site1.pkl')
frame2.to_pickle('./utilization_data_site2.pkl')